In [1]:
%load_ext autoreload
%autoreload 2
# Basic import:
import os
import cv2
import sys
import glob
import random
import zipfile
import numpy as np
import pandas as pd
from datetime import date
import matplotlib.pyplot as plt
from multiprocessing import cpu_count
import create_arrow
plt.ion() 
# Pytorch import:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from dataset import *
from models import *

In [2]:
batch_size = 256
classes = ['road', 'sidewalk', 'terrain', 'person', 'car']
# Get trainset and testset:
trainset = BDDSCAPES('./BDD', 'train', 
                     classes=classes,
                     new_size=(1440, 720), 
                     crop_size=(224, 448), 
                     random_resize=True,
                     is_flip=True,
                     to_tensor=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
unet = UNet(in_channels=3, 
            out_channels=len(trainset.classes)+1, 
            init_features=16, 
            bias=True)
unet = nn.DataParallel(unet)
unet.load_state_dict(torch.load('./saved_models/25.06.20_unet_61_val_nll=-0.20364859596888224.pth'))
unet.to(device)
unet.eval()

DataParallel(
  (module): UNet(
    (encoder1): Sequential(
      (enc1conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc1norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu1): ReLU(inplace=True)
      (enc1conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc1norm2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc1relu2): ReLU(inplace=True)
    )
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (encoder2): Sequential(
      (enc2conv1): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc2norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (enc2relu1): ReLU(inplace=True)
      (enc2conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (enc2norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=T

In [3]:
def fragment(img, n, channel_first=False):
    h, w, c = img.shape
    step_h = h // n
    step_w = w // n
    res = []
    for i in range(0, h-step_h+1, step_h):
        for j in range(0, w-step_w+1, step_w):
            if channel_first:
                res.append(img[i:i+step_h, j:j+step_w].permute(2, 0, 1))
            else:
                res.append(img[i:i+step_h, j:j+step_w])
    return res

def retrieve(img, n):
    h, w = img[0].shape
    res = np.zeros((h*n, w*n))
    for i in range(n):
        for j in range(n):
            h_pos, w_pos = i*h, j*w
            res[h_pos:h_pos+h, w_pos:w_pos+w] = img[i*n+j]
    return res

In [ ]:
validset = BDDSCAPES('./BDD', 'val',
                     classes=classes,
                     new_size=(320, 160), 
                     random_resize=False,
                     is_flip=False,
                     to_tensor=True)
#bdd = BDD(new_size=(640, 320))
cap = cv2.VideoCapture(1)
n = 1
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Our operations on the frame come here
    seg = validset.resize(frame)
    seg = validset.input_transform(seg)
    seg = torch.from_numpy(seg).to(device)
    seg = torch.stack(fragment(seg, n, True))
    with torch.no_grad():
        seg = unet(seg)
    seg = seg.argmax(dim=1).cpu().numpy()
    seg = retrieve(seg, n)
    seg = validset.label_to_color(seg)
    seg = seg[:,:,::-1]
    seg = cv2.resize(seg, (720, 360), interpolation=cv2.INTER_AREA)
    seg = create_arrow.paint_path(seg)
    # Display the resulting frame
    cv2.imshow('frame',seg)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()